<a href="https://colab.research.google.com/github/pulipulichen/colab-File-Browser/blob/main/colab/colab-File-Browser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **colab-File-Browser**

Instruction: https://sites.google.com/view/howtousecolab-colab-20250202-2


# Script

## Configurations for script

In [103]:
PROJECT_OWNER="pulipulichen"
PROJECT_NAME="colab-File-Browser"

FORCE_RUN_IN_FOREGROUND=False

### Testing Instruction

1. `Runtime` > `Run all` (Ctrl + F9)
2. Wait for the Cloudflare URL to be accessible.

## Core Tools

### curl_local_url()

In [104]:
import os
import time
import subprocess

inited = True
if os.path.isfile('.inited') is False:
  inited = False

def curl_local_url(port):
  while True:
    try:
      output = subprocess.check_output(['curl', '-s', 'http://127.0.0.1:' + str(port) + '/']).decode('utf-8')
      print("Success to connect http://127.0.0.1:" + str(port))
      break  # Break out of the loop if successful
    except subprocess.CalledProcessError:
      # print("Check http://127.0.0.1:" + str(port) + "/ failed. Retrying in 5 seconds...")
      time.sleep(5)
      continue  # Retry in case of failure

### wait_for_cloudflare()

In [105]:
import os
import time

def wait_for_cloudflare():
  file_path = f'/content/docker-app/{PROJECT_NAME}/.cloudflare.url'

  while not os.path.exists(file_path):
    time.sleep(3)  # Check every 1 second

  # File found, read and print its contents
  # with open(file_path, 'r') as file:
  #   content = file.read()
  #   print(f"Public URL: {content}")


### wait_for_docker_web_ready()

In [106]:
import os
import time

def wait_for_docker_web_ready():
  print('Waiting for docker web ready...')
  file_path = f'/content/docker-app/{PROJECT_NAME}/.docker-web.ready'

  while not os.path.exists(file_path):
    time.sleep(3)  # Check every 1 second

  # File found, read and print its contents
  # with open(file_path, 'r') as file:
  #   content = file.read()
  #   print(f"Public URL: {content}")

  time.sleep(10)

### keep_waiting()

In [107]:
import time

def keep_waiting():
  while True:
    time.sleep(1)  # Check every 1 second


### mountGDrive()

In [108]:
import os
from subprocess import getoutput
from google.colab import drive
import subprocess

GDRIVE_PATH = "/colab/" + PROJECT_NAME

def mountGDrive():
  if not os.path.exists("/google-drive/MyDrive"):
    return True

  if not os.path.exists("/google-drive"):
    drive.mount('/google-drive')  # access drive

  TEMP_GDRIVE_PATH = GDRIVE_PATH

  if not TEMP_GDRIVE_PATH.startswith("/"):
    TEMP_GDRIVE_PATH = "/" + TEMP_GDRIVE_PATH
  if not TEMP_GDRIVE_PATH.endswith("/"):
    TEMP_GDRIVE_PATH = TEMP_GDRIVE_PATH + "/"

  folder_path = "/google-drive/MyDrive/docker-app" + TEMP_GDRIVE_PATH
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)

  target_folder_path = "/content/docker-app/"
  if not os.path.exists(target_folder_path):
    os.makedirs(target_folder_path)

  command = ["ln", "-s", folder_path, target_folder_path]
  subprocess.run(command, check=True)

### clearDockerData()

In [109]:
import os
from subprocess import getoutput
import subprocess

def clearDockerData():
  stop_udocker()
  command = ["rm", "-rf", "/content/docker-app/" + PROJECT_NAME]
  subprocess.run(command, check=True)


### get_config_from_yaml()

In [110]:
import yaml
import subprocess
import time

if inited is False:
  !pip install CherryPy==18.8.0
  import cherrypy
  !pip install udocker==1.3.10
  !udocker --allow-root install

  from google.colab.output import eval_js
  !rm -rf ./sample_data*

  !pip install pyyaml==6.0.1

def get_config_from_yaml():
  yaml_file_path = f"./.docker-app/{PROJECT_NAME}/app-build/docker-compose-template.yml"

  with open(yaml_file_path, "r") as file:
    yaml_data = yaml.safe_load(file)
  # print(yaml_data)

  # Assuming there is only one service named "app" in your YAML
  webapp_ports = yaml_data.get("services", {}).get("app", {}).get("ports", [])
  WEBAPP_PORT = None

  for port_mapping in webapp_ports:
      # Check if the mapping is in the format "HOST_PORT:CONTAINER_PORT"
      if isinstance(port_mapping, str) and ":" in port_mapping:
        # print(port_mapping)
        host_port, container_port = port_mapping.split(":")
        WEBAPP_PORT = host_port
        break

  # ============
  environments = yaml_data.get("services", {}).get("app", {}).get("environment", [])

  # ============
  RUN_COMMAND = yaml_data.get("services", {}).get("app", {}).get("command", [])
  if RUN_COMMAND is not None:
    RUN_COMMAND = " ".join(RUN_COMMAND)

  # ============

  volumes = yaml_data.get("services", {}).get("app", {}).get("volumes", [])
  LOCAL_VOLUMN_PATH = '/data'

  for volumes_mapping in volumes:
      # Check if the mapping is in the format "HOST_PORT:CONTAINER_PORT"
      if isinstance(volumes_mapping, str) and ":" in port_mapping:
        # print(port_mapping)
        host_path, container_path = volumes_mapping.split(":")
        LOCAL_VOLUMN_PATH = container_path
        break

  # ============
  deploy = yaml_data.get("services", {}).get("app", {}).get("deploy", [])
  GPU_ENABLE = False
  if 'resources' in deploy:
    GPU_ENABLE = True

  # ============
  app = yaml_data.get("services", {}).get("app", {})
  # print(app)
  IMAGE_NAME = app['image']

  return {
    "WEBAPP_PORT": WEBAPP_PORT,
    "LOCAL_VOLUMN_PATH": LOCAL_VOLUMN_PATH,
    "IMAGE_NAME": IMAGE_NAME,
    "RUN_COMMAND": RUN_COMMAND,
    "GPU_ENABLE": GPU_ENABLE
  }


#### get_environments_from_yaml()

In [111]:
def get_environments_from_yaml(environments, key, default_value = None):

  for env_mapping in environments:
      # Check if the mapping is in the format "HOST_PORT:CONTAINER_PORT"
      if isinstance(env_mapping, str) and key + "=" in env_mapping:
        # print(port_mapping)
        key, value = env_mapping.split("=")
        default_value = value
        break

  return default_value

### get_current_datestring()

In [112]:
from datetime import datetime
def get_current_datestring():
  return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

## udocker

### stop_udocker()

In [113]:
def stop_udocker():
  #!udocker --allow-root ps
  !udocker --allow-root ps | awk 'NR > 1 {print $1}' | xargs -I {} udocker --allow-root rm {}
  !pgrep java && pkill java && sleep 5


### run_udocker()

In [114]:
def run_udocker():
  # WEBAPP_PORT, LOCAL_VOLUMN_PATH, IMAGE_NAME, RUN_COMMAND = get_config_from_yaml()
  YAML_CONFIG = get_config_from_yaml()

  # !rm -f /content/docker-app/$PROJECT_NAME/.docker-web.ready || true
  # !rm -f /content/docker-app/$PROJECT_NAME/.cloudflare.url || true
  ![ -e /content/docker-app/$PROJECT_NAME/.cloudflare.url ] && rm /content/docker-app/$PROJECT_NAME/.cloudflare.url
  ![ -e /content/docker-app/$PROJECT_NAME/.docker-web.ready ] && rm /content/docker-app/$PROJECT_NAME/.docker-web.ready

  RUN_IN_BACKGROUND = True
  if YAML_CONFIG["WEBAPP_PORT"] is None:
    RUN_IN_BACKGROUND = False

  # For testing purposes
  if FORCE_RUN_IN_FOREGROUND is True:
    RUN_IN_BACKGROUND = False

  if RUN_IN_BACKGROUND is False:
    run_udocker_in_foreground()
  else:
    run_udocker_in_background()


### run_udocker_in_foreground()

In [115]:
def run_udocker_in_foreground():

  # WEBAPP_PORT, LOCAL_VOLUMN_PATH, IMAGE_NAME, RUN_COMMAND = get_config_from_yaml()
  YAML_CONFIG = get_config_from_yaml()

  # print('Run container in foreground..')

  # print(LOCAL_VOLUMN_PATH)
  # print(RUN_COMMAND)

  !rm -rf /content/docker-app/$PROJECT_NAME
  !mkdir -p /content/docker-app/$PROJECT_NAME

  udocker_command = get_udocker_run_command()
  !$udocker_command

  print("Finish")

### run_udocker_in_background()

In [116]:
from datetime import datetime

def run_udocker_in_background():

  if os.path.isfile('.cloudflared') is False:
    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O .cloudflared
    !chmod a+x .cloudflared

  stop_udocker()

  !mkdir -p /content/docker-app/$PROJECT_NAME

  # WEBAPP_PORT, LOCAL_VOLUMN_PATH, IMAGE_NAME, RUN_COMMAND = get_config_from_yaml()
  YAML_CONFIG = get_config_from_yaml()

  !rm -rf ./*nohup.out
  !rm -rf ./docker-app/$PROJECT_NAME/.cloudflare.url

  print('Run container in background..')
  ![ -e /content/.docker-app/udocker.sh ] && rm /content/.docker-app/udocker.sh
  udocker_command = get_udocker_run_command()
  !echo "nohup $udocker_command > .nohup.out 2>&1 &" >> /content/.docker-app/udocker.sh
  !chmod +x /content/.docker-app/udocker.sh
  !bash /content/.docker-app/udocker.sh

  cherrypy.config.update({'server.socket_host': '0.0.0.0','server.socket_port' : int(YAML_CONFIG["WEBAPP_PORT"])})
  # wait_for_cloudflare()
  curl_local_url(YAML_CONFIG["WEBAPP_PORT"])
  wait_for_docker_web_ready()

  # =======

  ready_time = datetime.now()
  print('\nPreparing link... ' + ready_time.strftime("%Y-%m-%d %H:%M:%S"))

  # Calculate the difference in minutes
  interval_minutes = (ready_time - START_TIME).total_seconds() / 60
  print(f"Interval in minutes: {interval_minutes}")

  # =======

  print('\n============================================================================')
  print('請找到下面訊息的網址。網址的結尾必須為「.trycloudflare.com 」。')
  print('例如：https://organised-norton-talks-autos.trycloudflare.com')
  print('')
  print('如果超過5分鐘仍然沒有顯示資訊，請中斷執行後，重新執行這個cell。')
  print('============================================================================')
  #!cat /content/docker-app/docker-web-Apache-Solr/solrconfig.xml | grep dc.publisher
  cloudflared_command = './.cloudflared --url "http://127.0.0.1:' + YAML_CONFIG["WEBAPP_PORT"] + '"'
  !$cloudflared_command


### get_udocker_run_command()

In [117]:
from datetime import datetime

def get_udocker_run_command():
  YAML_CONFIG = get_config_from_yaml()

  command = ''

  IMAGE_NAME = YAML_CONFIG["IMAGE_NAME"]

  print('\nPulling the image ' + IMAGE_NAME + '...' + get_current_datestring())
  !udocker --allow-root pull $IMAGE_NAME

  print('\nCreate the name space for ' + IMAGE_NAME + ' ...' + get_current_datestring())
  !udocker --allow-root create --name=sdw $IMAGE_NAME

  if YAML_CONFIG["GPU_ENABLE"] == True:

    print('\nSetup nvidia for the name space sdw...' + get_current_datestring())
    !udocker --allow-root setup --nvidia sdw

  command = "udocker --allow-root run " + \
    "-p " + YAML_CONFIG["WEBAPP_PORT"] + ":" + YAML_CONFIG["WEBAPP_PORT"] + \
    " " + \
    "--volume=/content/docker-app/" + PROJECT_NAME + ":" + YAML_CONFIG["LOCAL_VOLUMN_PATH"] + \
    " " + \
    "sdw"

  print(command)

  return command

### setup_git_project()

In [118]:
def setup_git_project():
  if os.path.isdir('/content/.docker-app/') is False:
    !mkdir -p ./.docker-app

  if os.path.isdir('/content/.docker-app/' + PROJECT_NAME) is False:
    print('Try to initialize project...')

    %cd /content/.docker-app
    !git clone "https://github.com/{PROJECT_OWNER}/{PROJECT_NAME}.git"
    !git config --global pull.rebase false
  else:
    print('Try to update project...')

    %cd /content/.docker-app/$PROJECT_NAME
    !git reset --hard
    !git pull --force

  %cd /content/

## main() (must be the last)

In [119]:

if inited is False:
  %cd /content
  !touch .inited
  inited = True

# ==================

from datetime import datetime
START_TIME = datetime.now()

def main():
  setup_git_project()

  START_TIME = datetime.now()
  print('\nPreparing environment... ' + START_TIME.strftime("%Y-%m-%d %H:%M:%S"))

  # mountGDrive() # 開啟Google Drive掛載功能
  run_udocker()

In [120]:
# For Testing purposes

#clearDockerData()

# **Runtime**

In [121]:
# mountGDrive() # 開啟Google Drive掛載功能
main()

Try to update project...
/content/.docker-app/colab-File-Browser
HEAD is now at 96f3bcd Created using Colab
Committer identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@d7438626bb23.(none)')
/content

Preparing environment... 2025-02-03 13:16:37
Info: deleting container: a0cfed6d-2e56-36d7-9c22-31750fc92796
Run container in background..

Pulling the image pudding/docker-web:colab-file-browser-20250202.220343...2025-02-03 13:16:38
Info: downloading layer sha256:a26af6de5f791e11db1b79daa044d53d5dacb458925aec53b2da8976e6818f10
Info: downloading layer sha256:4f4fb700ef54461cfa02571ae0db9a0dc1e0cdb5577484a6d75e68dc38e8acc1
Info: downloading layer sha256:22a000724d336b2928d092eb5c6706338a1ec497f1c65457557fefb5d8a3a79b
Info: downloading layer